In [ ]:
# import cv2
# import mediapipe as mp

In [ ]:
# mp_hand = mp.solutions.hands#Landmarks
# mp_draw = mp.solutions.drawing_utils#drawing landmarks

In [ ]:
# data = mp_hand.Hands(1)

In [1]:
import cv2

In [3]:
import cv2
import mediapipe as mp
import pyautogui
import mss
import mss.tools
import ctypes
import tkinter as tk
import keyboard
import time
import sys

# --- STEP 1: DPI ACCURACY ---
try:
    ctypes.windll.shcore.SetProcessDpiAwareness(1)
except Exception:
    ctypes.windll.user32.SetProcessDPIAware()

# --- STEP 2: OVERLAY UI ---
class Overlay:
    def __init__(self):
        self.root = tk.Tk()
        self.root.overrideredirect(True)
        self.root.attributes("-topmost", True)
        self.root.attributes("-transparentcolor", "white")
        self.root.config(bg='white')
        self.root.wm_attributes("-disabled", True)
        
        self.canvas = tk.Canvas(self.root, width=self.root.winfo_screenwidth(), 
                               height=self.root.winfo_screenheight(), 
                               bg='white', highlightthickness=0)
        self.canvas.pack()
        self.rect = self.canvas.create_rectangle(0, 0, 0, 0, outline="green", width=3)
        
    def update_box(self, x1, y1, x2, y2, flash_now=False):
        try:
            self.canvas.coords(self.rect, x1, y1, x2, y2)
            if flash_now:
                self.canvas.itemconfig(self.rect, fill="white")
            else:
                self.canvas.itemconfig(self.rect, fill="")
            self.root.update()
        except:
            pass

    def close(self):
        try:
            self.root.destroy()
        except:
            pass

overlay = Overlay()

# --- STEP 3: COORDINATES & FLAGS ---
sct = mss.mss()
# Use monitor 1, but ensure it exists
try:
    main_monitor = sct.monitors[1]
except IndexError:
    main_monitor = sct.monitors[0]

current_box = [0, 0, 0, 0] 
capture_requested = False  
is_running = True 

def request_capture():
    global capture_requested
    capture_requested = True

def stop_program():
    global is_running
    is_running = False

# Register Hotkeys
keyboard.add_hotkey('s', request_capture)
keyboard.add_hotkey('q', stop_program)


# --- STEP 4: VISION BRAIN ---
screen_w, screen_h = pyautogui.size()
video = cv2.VideoCapture(0)
mp_hand = mp.solutions.hands
hands_detector = mp_hand.Hands(max_num_hands=1, min_detection_confidence=0.8)

p_t_x, p_t_y, p_i_x, p_i_y = 0, 0, 0, 0
smooth_factor = 0.15

print("AR Sniper Fixed & Ready.")

try:
    while True:
        # FIX: Global declarations must be at the very top of the loop
        global capture_requested, is_running
        
        if not is_running:
            break
            
        success, img = video.read()
        if not success: 
            break
        
        img = cv2.flip(img, 1)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = hands_detector.process(img_rgb)

        # 1. Update Box Coordinates
        if result.multi_hand_landmarks:
            for hand_lms in result.multi_hand_landmarks:
                thumb = hand_lms.landmark[4]
                index = hand_lms.landmark[8]
                
                p_t_x += (thumb.x * screen_w - p_t_x) * smooth_factor
                p_t_y += (thumb.y * screen_h - p_t_y) * smooth_factor
                p_i_x += (index.x * screen_w - p_i_x) * smooth_factor
                p_i_y += (index.y * screen_h - p_i_y) * smooth_factor

                current_box = [int(min(p_t_x, p_i_x)), int(min(p_t_y, p_i_y)), 
                               int(max(p_t_x, p_i_x)), int(max(p_t_y, p_i_y))]
        else:
            current_box = [-100, -100, -100, -100]

        # 2. Check for Capture Request
        if capture_requested:
            overlay.update_box(*current_box, flash_now=True)
            
            x1, y1, x2, y2 = current_box
            width, height = x2 - x1, y2 - y1
            
            if width > 20 and height > 20:
                region = {
                    "top": max(0, main_monitor["top"] + y1), 
                    "left": max(0, main_monitor["left"] + x1), 
                    "width": width, 
                    "height": height
                }
                sct_img = sct.grab(region)
                filename = f"cap_{int(time.time())}.png"
                mss.tools.to_png(sct_img.rgb, sct_img.size, output=filename)
                print(f"Captured: {filename}")
            
            time.sleep(0.1) 
            capture_requested = False 
        else:
            overlay.update_box(*current_box, flash_now=False)

        time.sleep(0.01)
except Exception as e:
    print(f"An error occurred: {e}")

finally:
    print("Shutting down and releasing camera...")
    video.release()
    overlay.close()
    # Unhook all hotkeys to prevent background listening after exit
    keyboard.unhook_all()
    sys.exit()

AR Sniper Fixed & Ready.
Captured: cap_1768589457.png

'Q' pressed. Shutting down...
Shutting down and releasing camera...


SystemExit: 